In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
! pip install ucimlrepo

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
air_quality = fetch_ucirepo(id=360) 
  
# data (as pandas dataframes) 
X = air_quality.data.features 
y = air_quality.data.targets 
  
# metadata 
print(air_quality.metadata) 
  
# variable information 
print(air_quality.variables) 


{'uci_id': 360, 'name': 'Air Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/360/air+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/360/data.csv', 'abstract': 'Contains the responses of a gas multisensor device deployed on the field in an Italian city. Hourly responses averages are recorded along with gas concentrations references from a certified analyzer. ', 'area': 'Computer Science', 'tasks': ['Regression'], 'characteristics': ['Multivariate', 'Time-Series'], 'num_instances': 9358, 'num_features': 15, 'feature_types': ['Real'], 'demographics': [], 'target_col': None, 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C59K5F', 'creators': ['Saverio Vito'], 'intro_paper': {'title': 'On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario', 'authors': 'S. D. Vito, E. Massera, M. P

In [3]:
df=X
df['Date_Time']=df['Date']+" "+df['Time']
df['Date_Time'] = pd.to_datetime(df.Date_Time)
data = df.drop(['Date_Time','Date','Time'], axis=1)
data.index = df.Date_Time

In [4]:
print(data.head())

                     CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  \
Date_Time                                                                     
2004-03-10 18:00:00     2.6         1360       150      11.9           1046   
2004-03-10 19:00:00     2.0         1292       112       9.4            955   
2004-03-10 20:00:00     2.2         1402        88       9.0            939   
2004-03-10 21:00:00     2.2         1376        80       9.2            948   
2004-03-10 22:00:00     1.6         1272        51       6.5            836   

                     NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  \
Date_Time                                                           
2004-03-10 18:00:00      166          1056      113          1692   
2004-03-10 19:00:00      103          1174       92          1559   
2004-03-10 20:00:00      131          1140      114          1555   
2004-03-10 21:00:00      172          1092      122          1584   
2004-03-10 22:00:00      131    

In [5]:
cols = data.columns
for j in cols:
    for i in range(0,len(data)):
        if data[j][i] == -200:
            data[j][i] = data[j][i-1]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12800\4041301214.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[j][i] = data[j][i-1]
C:\Users\Admin\AppData\Local\Temp\ipykernel_12800\4041301214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[j][i] = data[j][i-1]
C:\Users\Admin\AppData\Loc

In [6]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
# print(johan_test_temp.head())
coint_johansen(johan_test_temp,-1,1).eig

array([1.75510896e-01, 1.52389933e-01, 1.15120416e-01, 1.04126281e-01,
       9.29485509e-02, 6.89397159e-02, 5.77070988e-02, 3.43554214e-02,
       3.05980659e-02, 1.18697142e-02, 2.46766099e-03, 7.09584856e-05])

In [7]:
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [8]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [9]:
model = VAR(endog=train)
model_fit = model.fit()

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


In [10]:
prediction = model_fit.forecast(model_fit.endog, steps=len(valid))

In [11]:
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
        pred.iloc[i][j] = prediction[i][j]

C:\Users\Admin\AppData\Local\Temp\ipykernel_12800\537406477.py:4: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  pred.iloc[i][j] = prediction[i][j]


In [12]:
import math
from sklearn.metrics import mean_squared_error
for i in cols:
    print('rmse value for', i, 'is : ', math.sqrt(mean_squared_error(pred[i], valid[i])))

rmse value for CO(GT) is :  1.40868888368735
rmse value for PT08.S1(CO) is :  205.89558284021467
rmse value for NMHC(GT) is :  6.673548711372954
rmse value for C6H6(GT) is :  7.130087248704179
rmse value for PT08.S2(NMHC) is :  277.84844376794996
rmse value for NOx(GT) is :  214.78322340915082
rmse value for PT08.S3(NOx) is :  244.95769661939943
rmse value for NO2(GT) is :  66.69695211710696
rmse value for PT08.S4(NO2) is :  490.0838893410572
rmse value for PT08.S5(O3) is :  446.51541648880965
rmse value for T is :  10.721325795598325
rmse value for RH is :  17.111676248173076
rmse value for AH is :  0.5216247245183373


In [13]:
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.endog, steps=1)
print(yhat)

[[2.34596328e+00 1.08633212e+03 2.80762173e+02 1.24130779e+01
  1.05535947e+03 2.80882233e+02 6.59534851e+02 1.68444418e+02
  1.15918056e+03 8.50845529e+02 2.73639014e+01 1.55311062e+01
  5.15317053e-01]]


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)
